<a href="https://colab.research.google.com/github/nehashah2898/AIPI590/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import os, cv2, torch, numpy as np
from torch.utils.data import DataLoader, Dataset as BaseDataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler
import copy

CLASSES = ["background", "Solar_Panel", "Water_heater", "Pool_heater"]
COLORS = {0:(0,0,0), 1:(0,255,0), 2:(255,0,0), 3:(0,0,255)}

CKPT_PATH = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/pv-model-epoch=38-valid_avg_PV_iou=0.9572.ckpt"
IMG_DIR   = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/Test_Images_Prd"
OUT_BASE  = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts"

class PVModel(pl.LightningModule):
    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )
        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.register_buffer("std", torch.tensor(params["std"]).view(1,3,1,1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1,3,1,1))
    def forward(self, image):
        image = (image - self.mean) / self.std
        return self.model(image)

def get_validation_augmentation():
    return A.Compose([
        A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.CenterCrop(height=320, width=320),
        ToTensorV2(),
    ])

class InferenceDataset(BaseDataset):
    def __init__(self, image_dir, augmentation=None):
        self.image_paths = [os.path.join(image_dir,f) for f in os.listdir(image_dir)]
        self.augmentation = augmentation
    def __len__(self): return len(self.image_paths)
    def __getitem__(self,i):
        img_path = self.image_paths[i]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image.shape[:2] != (320,320):
            image = cv2.resize(image,(320,320))
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample["image"]
        else:
            image = torch.from_numpy(image.transpose(2,0,1)).float()
        return image, str(img_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = PVModel.load_from_checkpoint(
    CKPT_PATH,
    arch="FPN",
    encoder_name="resnext50_32x4d",
    in_channels=3,
    out_classes=len(CLASSES),
    strict=False
).to(device)
model.eval()

dataset = InferenceDataset(IMG_DIR, augmentation=get_validation_augmentation())
loader  = DataLoader(dataset, batch_size=1, shuffle=False)

target_layer = model.model.encoder.layer4[-1].conv3
print("Model and dataset are ready.")

ModuleNotFoundError: No module named 'pytorch_lightning'